In [1]:
from sklearn.cluster import KMeans
from news_classes import *
import math
from textblob import TextBlob as tb
import os
import pickle
import numpy as np

/usr/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/usr/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/usr/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/usr/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [2]:
idf_path='bdnews_idfblob_19_18.pkl'
with open(idf_path, 'rb') as input:
    idfblob = pickle.load(input)

In [3]:
def tf(word, blob):
    return blob.words.count(word) / len(blob.words)

def idf(word):
    return math.log(idfblob.size() / (1 + idfblob.getIdf(word)))

def tfidf(word, blob):
    return tf(word, blob) * idf(word)

def sentence_tfidf(sent_blob, para_blob):    
    tot=0;
    for word in sent_blob.words:
        tot+=tfidf(word,para_blob)
    
    return tot/len(sent_blob.words)

In [4]:
def get_sent_idfs(para):
    sentences=tb(para).lower().sentences
        
    sent_tfidf=[]
    for sentence in sentences:
        sent_tfidf.append(sentence_tfidf(sentence, tb(para).lower()))

    return sent_tfidf

In [67]:
def summarize(para):
    sent_idfs=get_sent_idfs(para)
    
    X1 = np.array(sent_idfs)
    X1=X1.reshape(-1,1)
    X2 = np.zeros((len(X1),1))
    X = np.hstack((X1,X2))
    
    cluster_size=len(X)//2
    kmeans = KMeans(n_clusters=cluster_size, random_state=0).fit(X)
    
    counter={}
    n=len(X)
    for i in range(cluster_size):
        counter[i]=0
    for i in range(n):
        counter[kmeans.labels_[i]]+=1
    mx=0
    for i in range(cluster_size):
        if counter[i]>counter[mx]:
            mx=i
            
    sentences=tb(para).sentences
    summarized_para=""
    for i in range(len(sentences)):
        if kmeans.labels_[i] == mx:
            summarized_para+=(str(sentences[i])+" ")
    
    return summarized_para

In [68]:
para="An investigation committee had been formed when the allegation had surfaced and the committee submitted its report to the disciplinary body, Proctor AKM Golam Rabbani told bdnews24.com on Monday. The Syndicate decided to suspend the seven students following recommendations by the disciplinary board in a meeting on July 10, the proctor said. “The decisions have been conveyed to the related departments and halls,” he added. Khairun Nisa of 2014-15 session has been suspended for one year. The others got six-month suspension each. They are Umme Habiba Tanzila of 2014-15 session, Minhazul Abedin, Samira Mahzabin and ‘Fatima’ of 2015-16, and Abdullah Hil Baki and Aditi Sarker of 2016-17. "
print(summarize(para))

An investigation committee had been formed when the allegation had surfaced and the committee submitted its report to the disciplinary body, Proctor AKM Golam Rabbani told bdnews24.com on Monday. The others got six-month suspension each. 
